# ***FilmFusión: Sistema de Recomendación de Películas***

### FilmFusión es un sistema de recomendación de películas basado en inteligencia artificial que ayuda a los usuarios a encontrar la película ideal según sus gustos y preferencias. Utilizando técnicas de aprendizaje automático y análisis de datos, FilmFusión ofrece recomendaciones personalizadas al analizar información sobre películas, como géneros, actores, valoraciones y descripciones. El sistema puede ser implementado utilizando bibliotecas populares de Python como Pandas, NumPy y Scikit-learn, permitiendo a los usuarios descubrir nuevas películas que se alineen con sus intereses. Con FilmFusión, la búsqueda de la próxima gran película se convierte en una experiencia intuitiva y emocionante.

* Para este proyecto estaremos utilizando una data ya existente que la sacamos de https://grouplens.org/datasets/movielens/, son calificaciones de películas de MovieLens 100K . Conjunto de datos de referencia estable. 100 000 calificaciones de 1000 usuarios sobre 1700 películas. Publicado en abril de 1998.

# **1. Importación de librerias**

In [58]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import StandardScaler

# **2. Carga de Data**

In [59]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [60]:
import pandas as pd

# Cambia 'ruta/a/tu/carpeta' por la ruta real en tu Google Drive
data_path = '/content/drive/My Drive/dataproyecto2/'

# Cargar las calificaciones
ratings = pd.read_csv(data_path + 'u.data', sep='\t', header=None, names=['user_id', 'item_id', 'rating', 'timestamp'])

# Cargar la información de las películas
movies = pd.read_csv(data_path + 'u.item', sep='|', header=None, encoding='latin-1',
                        names=['item_id', 'title', 'release_date', 'video_release_date', 'IMDb_URL'] + ['genre_' + str(i) for i in range(19)])

# Cargar la información de los usuarios
users = pd.read_csv(data_path + 'u.user', sep='|', header=None, names=['user_id', 'age', 'gender', 'occupation', 'zip_code'])

# Mostrar las primeras filas de cada DataFrame
print(ratings.head())
print(movies.head())
print(users.head())

   user_id  item_id  rating  timestamp
0      196      242       3  881250949
1      186      302       3  891717742
2       22      377       1  878887116
3      244       51       2  880606923
4      166      346       1  886397596
   item_id              title release_date  video_release_date  \
0        1   Toy Story (1995)  01-Jan-1995                 NaN   
1        2   GoldenEye (1995)  01-Jan-1995                 NaN   
2        3  Four Rooms (1995)  01-Jan-1995                 NaN   
3        4  Get Shorty (1995)  01-Jan-1995                 NaN   
4        5     Copycat (1995)  01-Jan-1995                 NaN   

                                            IMDb_URL  genre_0  genre_1  \
0  http://us.imdb.com/M/title-exact?Toy%20Story%2...        0        0   
1  http://us.imdb.com/M/title-exact?GoldenEye%20(...        0        1   
2  http://us.imdb.com/M/title-exact?Four%20Rooms%...        0        0   
3  http://us.imdb.com/M/title-exact?Get%20Shorty%...        0        1   

# **3. Limpieza y preprocesamiento de los datos**

### 3.1. Eliminar columnas innecesarias de movies

In [61]:
movies = movies[['item_id', 'title']]
print(movies.head())

   item_id              title
0        1   Toy Story (1995)
1        2   GoldenEye (1995)
2        3  Four Rooms (1995)
3        4  Get Shorty (1995)
4        5     Copycat (1995)


### 3.2. Convertir tipos de datos

In [62]:
ratings['timestamp'] = pd.to_datetime(ratings['timestamp'], unit='s')
print(ratings.head())

   user_id  item_id  rating           timestamp
0      196      242       3 1997-12-04 15:55:49
1      186      302       3 1998-04-04 19:22:22
2       22      377       1 1997-11-07 07:18:36
3      244       51       2 1997-11-27 05:02:03
4      166      346       1 1998-02-02 05:33:16


### 3.3. Manejo de valores faltantes

In [63]:
print(ratings.isnull().sum())
print(movies.isnull().sum())
print(users.isnull().sum())

user_id      0
item_id      0
rating       0
timestamp    0
dtype: int64
item_id    0
title      0
dtype: int64
user_id       0
age           0
gender        0
occupation    0
zip_code      0
dtype: int64


### 3.4. Normalizar los datos

In [64]:
#calificación normalizada = calificación - min / ma - min

# Definir el rango de las calificaciones
min_rating = 1
max_rating = 5

# Normalizar las calificaciones
ratings['normalized_rating'] = (ratings['rating'] - min_rating) / (max_rating - min_rating)

# Mostrar las primeras filas para verificar
print(ratings[['user_id', 'item_id', 'rating', 'normalized_rating']].head())


   user_id  item_id  rating  normalized_rating
0      196      242       3               0.50
1      186      302       3               0.50
2       22      377       1               0.00
3      244       51       2               0.25
4      166      346       1               0.00


# **4.  Crear una matriz de calificaciones:**

##### Crear una matriz donde las filas son usuarios y las columnas son películas. Esto facilitará el cálculo de similitudes.

# **5. Dividir el conjunto de datos:**

##### Dividimos los datos en conjuntos de entrenamiento y prueba para evaluar el sistema de recomendación

# **6. Calcular la similitud entre usuarios:**

##### Usamos la similitud del coseno para calcular qué tan similares son los usuarios en base a sus calificaciones.

# **7.  Función para hacer recomendaciones:**

##### Definimos una función que toma un ID de usuario y devuelve recomendaciones basadas en usuarios similares.

# **8. Probar el sistema de recomendación:**

##### Probamos la función con un usuario específico y mostramos las películas recomendadas.